In [ ]:
#Instal
%pip install yfinance --upgrade --no-cache-dir
# virtualenv -p python3 virtualenvironment
# source virtualenvironment/bin/activate
%pip install ta --upgrade --no-cache-dir
%pip install pandas_ta --upgrade --no-cache-dir
%pip install pandas-datareader --upgrade --no-cache-dir
%pip install matplotlib --upgrade --no-cache-dir

#imports
import datetime as dt
import sys
from math import floor

import numpy as np
import pandas as pd

import yfinance as yf
import pandas_datareader.data as web
import matplotlib.pyplot as plt

from ta.volatility import bollinger_mavg, KeltnerChannel

yf.pdr_override()

In [ ]:
#Define functions

#GET YAHOO DATA
def get_data(ticker, start, end):
    ticker_data = yf.download(ticker, start, end)
    return ticker_data

def convert_data(ticker_data, ticker):
    
    names = []
    table = []
    
    ticker_data = ticker_data.rename(columns = {ticker:'Close'})
    for i in range(0, LOOP_MAX, SHIFT_DAYS) :
        data = ticker_data[i:CORR_WINDOW + i]
        date = data.index[0]
        ticker_name = ticker + ' ' + date.strftime('%Y/%m/%d')
        names.append(ticker_name)
        data['Ticker'] = ticker_name
        table.append(data)    
        
    return [names, table]

#CREATE DATAFRAMES
def create_dfs(datas_to_df):
    dfs = []
    for data_to_df in datas_to_df :
        name, data = data_to_df
        df = pd.DataFrame()
        df = pd.concat(data)
        df = df.reset_index()
        df = df.pivot(columns='Ticker',values='Close')
        dfs.append(df)
    return dfs

#PIVOT FUNCTIONS
def pivot_df(df_pivot, data, names): 
    for i in range(0, LOOP_MAX, SHIFT_DAYS):
        column_name = names[i]
        values = data[column_name].values
        values = values[~np.isnan(values)]
        values = pd.Series(values)
        df_pivot.insert(len(df_pivot.columns), column_name, values)
        
    return df_pivot

def pivot_all_dfs(dfs, datas):
    dfs_pivoted = []
    names = []
    loop_times = len(dfs)
    
    for j in range(loop_times):
        for i in range(j+1, loop_times):
            name_to_first = datas[j][0]
            names_to_second = datas[i][0]
            names.append([name_to_first, names_to_second])
            
            df_pivoted = pd.DataFrame()
            df_pivoted = pivot_df(df_pivoted, dfs[j], name_to_first)
            df_pivoted = pivot_df(df_pivoted, dfs[i], names_to_second)
            dfs_pivoted.append(df_pivoted)
        
    return [dfs_pivoted, names]

#CORRELATION FUNCTIONS
def get_corr_df(df_pivot, nameRow, nameColumn):
    #Using corr function to get correlation between actives
    corr_df = df_pivot.corr(method='pearson')
    #reset symbol as index (rather than 0-X)
    corr_df.head().reset_index()
    corr_df = corr_df.rename_axis(None, axis=0)
    corr_df = corr_df.drop(nameRow, axis=1)
    corr_df = corr_df.drop(nameColumn, axis=0)
    
    return corr_df

def get_all_corr_dfs(dfs, names):
    dfs_corr = []
    for i in range(len(dfs)):
        corr_df = get_corr_df(dfs[i], names[i][0], names[i][1])
        dfs_corr.append(corr_df)
    
    return dfs_corr

#INDICATOR FUNCTIONS
def get_indicators(df):
    values = df.values
    means_df = pd.DataFrame()
    deviations_df = pd.DataFrame()
    columns_name = []
    rang = floor(LOOP_MAX - CORR_WINDOW)
    for i in range(rang):
        means = []
        deviations = []
        row_name = df.index[i] + " - " + df.index[i + CORR_WINDOW].split(' ')[1] 
        for j in range(rang):
            if i == 0:
                column_name = df.columns[j] + " - " + df.columns[j + CORR_WINDOW].split(' ')[1] 
                columns_name.append(column_name)
            ax = []
            for k in range(CORR_WINDOW):
                value = values[i+k][j+k]
                ax.append(value)

            mean = np.average(ax)
            means.append(mean)

            deviation = np.std(ax)
            deviations.append(deviation)
            
        aux_df = pd.DataFrame([means], columns=columns_name, index=[row_name])
        means_df = pd.concat([means_df, aux_df])
        aux_df = pd.DataFrame([deviations], columns=columns_name, index=[row_name])
        deviations_df = pd.concat([deviations_df, aux_df])
        
    return [means_df, deviations_df]

def get_df_plot(dfs_corr):
    means, deviations = get_indicators(dfs_corr)
    
    rang = len(means.index) - SHIFT_DAYS
    values_means = means.values
    values_deviations = deviations.values
    values = []
    columns = [1,2,3,4]
    
    for i in range(rang):
        value = []
        value_mean = values_means[i][i + SHIFT_DAYS]
        value_deviation = values_deviations[i][i + SHIFT_DAYS]
        value.append(value_mean)
        value.append(value_deviation)
        
        value_mean = values_means[i + SHIFT_DAYS][i]
        value_deviation = values_deviations[i + SHIFT_DAYS][i]
        value.append(value_mean)
        value.append(value_deviation)
        
        values.append(value)
    columns = get_columns_name(means)    
    final_df = pd.DataFrame(values, columns=columns)
    return final_df

def get_columns_name(df):
    names = []
    
    for ind in ["Means", "Deviations"]:
        ticker = df.columns[0].split(" ")[0]
        
        index = df.index[0].split(" ")
        start_date = index[1]

        last_index = df.index[len(df.index) - (SHIFT_DAYS + 1)].split(" ") 
        end_date = last_index[1]
        name = ind + " - "
        name = name + ticker + " start at "
        name = name + start_date + " unitl "
        name = name + end_date + "\n"

        index = df.index[1].split(" ")
        last_index = df.index[-1].split(" ")
        start_date = index[1]
        end_date = last_index[1]
        ticker = index[0]
        
        name = name + ticker + " start at "
        name = name + start_date + " unitl "
        name = name + end_date
        names.append(name)
        
    for ind in ["Means", "Deviations"]:
        index = df.index[0].split(" ")
        start_date = index[1]
        ticker = index[0]
        last_index = df.index[len(df.index) - (SHIFT_DAYS + 1)].split(" ") 
        end_date = last_index[1]
        
        name = ind + " - "
        name = name + ticker + " start at "
        name = name + start_date + " unitl "
        name = name + end_date + "\n"

        index = df.index[1].split(" ")
        last_index = df.index[-1].split(" ")
        start_date = index[1]
        end_date = last_index[1]
        ticker = df.columns[0].split(" ")[0]
        
        name = name + ticker + " start at "
        name = name + start_date + " unitl "
        name = name + end_date
        names.append(name)
        
    return names

def get_price_df(df_p, names):
    df = pd.DataFrame()
    dates = []
    length = len(df_p[names[0]].columns)
    
    column = []    
    ticker = names[0][0].split(" ")[0]
    for i in range(length):
        dates.append(names[0][i].split(" ")[1])
        column.append(df_p[names[0]].values[0][i])
    
    df.insert(len(df.columns), ticker, column)
    
    column = []
    ticker = names[1][0].split(" ")[0]
    for i in range(length):
        column.append(df_p[names[1]].values[0][i])
            
    df.insert(len(df.columns), ticker, column) 
    
    mavg = get_mavg(df)
    df = df.join(mavg)
    df.insert(0, "Date", dates)
    return df

def get_mavg(df, ticker):
    mavg_df = pd.DataFrame()
    
    mavg_aux = bollinger_mavg(df[ticker + "-Close"], window=MAVG_WINDOW, fillna=True)
    mavg_df.insert(0, ticker + "-MA", mavg_aux)
        
    return mavg_df

#Keltner Channel
def get_price_KC(data, ticker):    
    high = list(data["High"][ticker])
    close = list(data["Close"][ticker])
    low = list(data["Low"][ticker])
    
    df_price = pd.DataFrame({ticker + "-High": high, ticker + "-Close": close, ticker + "-Low": low})
    
    KC = KeltnerChannel(high=df_price[ticker + "-High"], 
                        close=df_price[ticker + "-Close"], 
                        low=df_price[ticker + "-Low"], 
                        multiplier=MULTIPLIER, window=20, fillna=True)
    
    return [KC, df_price]

def get_all_price_KC(data):
    lenght = len(data["High"].columns)
    dfs = []
    df_prices = []
    
    dates = list(data.index.strftime('%Y/%m/%d'))
    
    for i in range(lenght):
        ticker1 = tickers[i]
        df_kc1, df_price1 = get_price_KC(data, ticker1)
        high1 = list(df_kc1.keltner_channel_hband())
        low1 = list(df_kc1.keltner_channel_lband())
        mavg1 = get_mavg(df_price1, ticker1)
        df_price1 = df_price1.join(mavg1)
        
        for j in range(i+1, lenght):
            ticker2 = tickers[j]
            df_kc2, df_price2 = get_price_KC(data, ticker2)
            high2 = list(df_kc2.keltner_channel_hband())
            low2 = list(df_kc2.keltner_channel_lband())
            mavg2 = get_mavg(df_price2, ticker2)
            df_price2 = df_price2.join(mavg2)
            
            df_price = df_price1.join(df_price2)
            df_price.insert(0, "date", dates)
            df_prices.append(df_price)
            
            df_aux = pd.DataFrame({ticker1 + "-High": high1, ticker1 + "-Low": low1, 
                                   ticker2 + "-High": high2, ticker2 + "-Low": low2})
            dfs.append(df_aux)
            
    return [dfs, df_prices]

In [ ]:
def trade_comprado(corr_df, price_df, kcs_df, predict_stock, isTest: False):
    equity = pd.DataFrame(columns=['Date', 'Profit'])
    
    corr_length = len(corr_df.index)
    
    date = price_df.columns[0]
    dates = price_df[date]
    
    if(predict_stock == 1):
        stock_a = price_df.columns[2]
        stock_a_mavg = price_df.columns[4]
        
        stock_b_high = price_df.columns[5]
        stock_b = price_df.columns[6]
        stock_b_mavg = price_df.columns[8]

        stock_b_predicting_mean_corr = corr_df.columns[2]

        stock_b_high_kc = kcs_df.columns[2]
    
    else:
        stock_a = price_df.columns[6]
        stock_a_mavg = price_df.columns[8]
        
        stock_b_high = price_df.columns[1]
        stock_b = price_df.columns[2]
        stock_b_mavg = price_df.columns[4]

        stock_b_predicting_mean_corr = corr_df.columns[0]

        stock_b_high_kc = kcs_df.columns[0]
        
    if isTest:
        stock_b_predicting_mean_corr = corr_df.columns[2]
    
    file = open(path_comprado + stock_b + ".txt", "a+")
    
    nmb_orders = 0
    paid_stock = 0
    sell_cause = ""
    bought = False
    stop = 0.95
    
    title = "Usando {0} para prever {1}\n".format(stock_a.split("-")[0], stock_b.split("-")[0])
    file.write(title)

    for i in range(corr_length):
        stock_price = price_df[stock_b][i]
        high_stock_price = price_df[stock_b_high][i]
        profit_test = ((stock_price/paid_stock)-1)
        if bought:
            if high_stock_price > kcs_df[stock_b_high_kc][i]:
                sell_cause = "Better than KC"
                bought = False
            if price_df[stock_a][i] < price_df[stock_a_mavg][i]:
                sell_cause = "{0} is low than mavg".format(stock_a)
                bought = False
            # if profit_test > 2:
            #     stop = 1 + profit_test
            # if stock_price > paid_stock * stop:
            #     sell_cause = "Stop loss".format()
            #     bought = False
                    
            if not bought:
                profit = ((stock_price/paid_stock)-1)*100
                EC_aux = pd.DataFrame([{"Date": dates[i], "Profit": profit}])
                equity = pd.concat([equity, EC_aux])
                file.write("selling in {0} - stock price: {1} - cause:{2} - profit:{3}\n".format(dates[i], stock_price, sell_cause, profit))
                nmb_orders += 1
                stop = 0.95
        else:
            if price_df[stock_a][i] > price_df[stock_a_mavg][i]:
                if stock_price < price_df[stock_b_mavg][i]:
                    if corr_df[stock_b_predicting_mean_corr][i] > ENTER_CORR:
                        bought = True
                        paid_stock = stock_price
                        file.write("buying in {0} - stock price: {1}\n".format(dates[i], stock_price))
                        nmb_orders += 1
                        
    profits = equity["Profit"].sum()
    file.write("Profit in final of tests {0} and Number of orders {1}\n\n".format(profits, nmb_orders))
    
    file.close()
    stock_a = stock_a.split('-')[0]
    stock_b = stock_b.split('-')[0]
    return [equity, nmb_orders, [stock_a, stock_b]]

def trade_vendido(corr_df, price_df, kcs_df, predict_stock, isTest: False):
    equity = pd.DataFrame(columns=['Date', 'Profit'])
    
    corr_length = len(corr_df.index)
    
    date = price_df.columns[0]
    dates = price_df[date]
    
    if(predict_stock == 1):
        
        stock_a = price_df.columns[2]
        stock_a_mavg = price_df.columns[4]
        
        stock_b = price_df.columns[6]
        stock_b_low = price_df.columns[7]
        stock_b_mavg = price_df.columns[8]

        stock_b_predicting_mean_corr = corr_df.columns[2]

        stock_b_low_kc = kcs_df.columns[3]
    
    else:
        stock_a = price_df.columns[6]
        stock_a_mavg = price_df.columns[8]
        
        stock_b = price_df.columns[2]
        stock_b_low = price_df.columns[3]
        stock_b_mavg = price_df.columns[4]
        
        stock_b_predicting_mean_corr = corr_df.columns[0]
        
        stock_b_low_kc = kcs_df.columns[1]
    
    if isTest:    
        stock_b_predicting_mean_corr = corr_df.columns[2]
    
    file = open(path_vendido + stock_b + ".txt", "a+")
    
    nmb_orders = 0
    sold_stock = 0
    purchase_cause = ""
    sold = False
    stop = 1.05

    title = "Usando {0} para prever {1}\n".format(stock_a.split("-")[0], stock_b.split("-")[0])
    file.write(title)

    for i in range(corr_length):
        stock_price = price_df[stock_b][i]
        low_stock_price = price_df[stock_b_low][i]
        profit_test = ((sold_stock/stock_price)-1)
        if sold:
            if low_stock_price < kcs_df[stock_b_low_kc][i]:
                purchase_cause = "Lower than KC"
                sold = False
            if price_df[stock_a][i] > price_df[stock_a_mavg][i]:
                purchase_cause = "{0} is better than mavg".format(stock_a)
                sold = False
            # if profit_test > 2:
            #     stop = 1 - profit_test
            # if stock_price < sold_stock * stop:
            #     purchase_cause = "Stop loss".format()
            #     sold = False
            
            if not sold:
                profit = ((sold_stock/stock_price)-1)*100
                EC_aux = pd.DataFrame([{"Date": dates[i], "Profit": profit}])
                equity = pd.concat([equity,EC_aux])
                file.write("Buying in {0} - stock price: {1} - cause:{2} - profit:{3}\n".format(dates[i], stock_price, purchase_cause, profit))
                nmb_orders += 1
                stop = 1.05
                
        else:
            if price_df[stock_a][i] < price_df[stock_a_mavg][i]:
                if stock_price > price_df[stock_b_mavg][i]:
                    if corr_df[stock_b_predicting_mean_corr][i] > ENTER_CORR:
                        sold = True
                        sold_stock = stock_price
                        file.write("Selling in {0} - stock price: {1}\n".format(dates[i], stock_price))
                        nmb_orders += 1
                        
    profits = equity["Profit"].sum()
    file.write("Profit in final of tests {0} and Number of orders {1}\n\n".format(profits, nmb_orders))
        
    file.close()
    return [equity, nmb_orders]

In [ ]:
def new_parameters(profit, key, nmb_orders):

    new_parameters = {}
    new_parameters['PROFIT'] = profit
    new_parameters['CORR_WINDOW'] = CORR_WINDOW
    new_parameters['MAVG_WINDOW'] = MAVG_WINDOW
    new_parameters['ENTER_CORR'] = ENTER_CORR 
    new_parameters['ORDERS'] = nmb_orders
    
    global parameters
    parameters[key] = new_parameters
    return

def train():
    global parameters
    global path_comprado
    path_comprado = "trade_comprado_train"
    global path_vendido
    path_vendido = "trade_vendido_train"

    loaded_datas = get_data(tickers, START_DATE_TRAIN, END_DATE_TRAIN)
    for corr_window in [5,7,9]:
        global CORR_WINDOW
        CORR_WINDOW = corr_window
        global LOOP_MAX
        LOOP_MAX = len(loaded_datas) - CORR_WINDOW - SHIFT_DAYS

        all_data = []
        for ticker in tickers:
            data = convert_data(pd.DataFrame(loaded_datas["Close"][ticker]), ticker)
            all_data.append(data)
        
        dfs = create_dfs(all_data)
        dfs_pivoted, names = pivot_all_dfs(dfs, all_data)

        dfs_corr = get_all_corr_dfs(dfs_pivoted, names)
        
        len_dfs_corr = len(dfs_corr)
        range_len_dfs_corr = range(len_dfs_corr)
        
        indicators_dfs = []
        for x in range_len_dfs_corr:
            aux_ind = get_df_plot(dfs_corr[x])
            indicators_dfs.append(aux_ind)

        for mavg_window in [36,38,40,42]:
            global MAVG_WINDOW
            MAVG_WINDOW = mavg_window
            dfs_kc, prices_dfs = get_all_price_KC(loaded_datas)
            
            for enter_corr in [0.64,0.66,0.68,0.7,0.72]:
                global ENTER_CORR
                ENTER_CORR = enter_corr
                for i in range_len_dfs_corr:
                    for number in [1, 0]:
                        
                        equity_curve = pd.DataFrame(columns=['Date', 'Profit'])
                        
                        equity_aux, nmbs, stocks = trade_comprado(indicators_dfs[i], prices_dfs[i], dfs_kc[i], number, False)
                        nmb_orders = nmbs
                        equity_curve = pd.concat([equity_curve, equity_aux], ignore_index=True)
                        equity_aux, nmbs = trade_vendido(indicators_dfs[i], prices_dfs[i], dfs_kc[i], number, False)
                        nmb_orders += nmbs
                        equity_curve = pd.concat([equity_curve, equity_aux], ignore_index=True)

                        equity_curve = pd.DataFrame(equity_curve.groupby(['Date']).Profit.sum().cumsum())
                        list_equity_curve = list(equity_curve['Profit'].values)
                        final_profit = 0
                        final_profit = list_equity_curve[-1] if len(list_equity_curve) > 0 else 0
                        final_profit = round(final_profit, 3)
                        
                        key = 'Usando {0} para prever {1}'.format(stocks[0], stocks[1])
                        
                        if key not in parameters:
                            parameters[key] = {'PROFIT':-sys.maxsize,'CORR_WINDOW':0,'MAVG_WINDOW':0,'ENTER_CORR':0, 'ORDERS': 0}
                        
                        if final_profit > parameters[key]['PROFIT'] and final_profit != 0:
                            new_parameters(final_profit, key, nmb_orders)
    global dfParameters                            
    dfParameters = pd.DataFrame(parameters)
    dfParameters = dfParameters.replace([-sys.maxsize], 0)

In [ ]:
def test(start_date, end_date):
    
    global profit
    global path_comprado
    global path_vendido
    global len_loaded_datas
    
    profit = {}
    path_comprado = "trade_comprado_test"
    path_vendido = "trade_vendido_test"
    loaded_datas = get_data(tickers, start_date, end_date)
    len_loaded_datas = len(loaded_datas)
    
    count = 0
    aux_count = 0
    
    transposed = dfParameters.transpose()
    
    for i in range(len_tickers):
        for j in range(i+1, len_tickers):
            for k in [1,0]:
                param = transposed.values[count]
                count = count + 1
                
                global CORR_WINDOW
                global MAVG_WINDOW
                global ENTER_CORR
                global LOOP_MAX
                
                CORR_WINDOW = int(param[1])
                MAVG_WINDOW = int(param[2])
                ENTER_CORR = param[3]
                
                if(CORR_WINDOW == 0):
                    continue
                
                LOOP_MAX = len_loaded_datas - CORR_WINDOW - SHIFT_DAYS
                
                if k == 1:
                    ticker_a = tickers[i]
                    ticker_b = tickers[j]
                else:
                    ticker_a = tickers[j]
                    ticker_b = tickers[i]
                
                tics = [ticker_a, ticker_b]
                all_data = []
                for tic in tics:                
                    dfToConvert = pd.DataFrame(loaded_datas["Close"][tic])
                    data = convert_data(dfToConvert, tic)
                    all_data.append(data)
                
                dfs = create_dfs(all_data)
                
                dfs_pivoted, names = pivot_all_dfs(dfs, all_data)

                dfs_corr = get_all_corr_dfs(dfs_pivoted, names)
                
                indicators = get_df_plot(dfs_corr[0])
                
                dfs_kc, prices_dfs = get_all_price_KC(loaded_datas)
                
                equity_curve = pd.DataFrame(columns=['Date', 'Profit'])
                
                equity_aux, nmbs, stocks = trade_comprado(indicators, prices_dfs[aux_count], dfs_kc[aux_count], k, True)
                nmb_orders = nmbs
                equity_curve = pd.concat([equity_curve, equity_aux], ignore_index=True)
                equity_aux, nmbs = trade_vendido(indicators, prices_dfs[aux_count], dfs_kc[aux_count], k, True)
                nmb_orders += nmbs
                equity_curve = pd.concat([equity_curve, equity_aux], ignore_index=True)

                equity_curve = pd.DataFrame(equity_curve.groupby(['Date']).Profit.sum().cumsum())
                list_equity_curve = list(equity_curve['Profit'].values)
                final_profit = list_equity_curve[-1] if len(list_equity_curve) > 0 else 0
                final_profit = round(final_profit, 3)
                
                key = 'Usando {0} para prever {1}'.format(stocks[0], stocks[1])
                
                profit[key] = [final_profit, nmb_orders]
                equity_curve = equity_curve.rename(columns={'Profit': key})
                global ECs
                ECs.append(equity_curve)
                
            aux_count = aux_count + 1
            

In [ ]:
SHIFT_DAYS = 1

CORR_WINDOW = 1
MAVG_WINDOW = 1
ENTER_CORR = 1
LOOP_MAX = 1

MULTIPLIER = 1

parameters = {}

path_comprado = ''
path_vendido = ''

In [ ]:
# tickers = ['B3SA3.SA','SANB11.SA','ELET6.SA']#
tickers = ['SMTO3.SA','BEEF3.SA','MRFG3.SA','JBSS3.SA','BRFS3.SA']#
# tickers = ['VALE3.SA','PETR4.SA','ITUB4.SA','BBDC4.SA', 'B3SA3.SA','ABEV3.SA','SANB11.SA','ELET6.SA']#
# tickers = ['BBDC3.SA','BBAS3.SA','ITUB4.SA','B3SA3.SA','BPAC11.SA','SANB11.SA','ITSA4.SA','BBSE3.SA']#
# tickers = ['EGIE3.SA','ENEV3.SA','ENGI11.SA','ENBR3.SA','ELET3.SA','CPLE6.SA','CMIG4.SA','EQTL3.SA','TAEE11.SA']#

len_tickers = len(tickers)
range_len_tickers = range(len_tickers)

In [ ]:
START_DATE_TRAIN = '2021-1-1'
END_DATE_TRAIN = '2022-1-1'

START_DATE_TEST0 = '2021-1-1'
END_DATE_TEST0 = '2022-1-1'

START_DATE_TEST1 = '2022-1-1'
END_DATE_TEST1 = '2023-1-1'

# START_DATE_TEST2 = '2023-1-1'
# END_DATE_TEST2 = '2023-5-16'

# START_DATE_TEST3 = '2021-1-1'
# END_DATE_TEST3 = '2022-1-1'

# START_DATE_TEST4 = '2022-1-1'
# END_DATE_TEST4 = '2023-1-1'

ECs = []

# START_DATE_TRAIN = '2017-1-1'
# END_DATE_TRAIN = '2018-1-1'
# START_DATE_TEST = '2018-1-1'
# END_DATE_TEST = '2019-1-1'

train()

test(START_DATE_TEST0, END_DATE_TEST0)
alo = pd.DataFrame(profit.values(), profit.keys(), columns=["Control profit/" + START_DATE_TEST0, "Controle operações"]).T
concated = pd.concat([ dfParameters, alo ])

test(START_DATE_TEST1, END_DATE_TEST1)
alo = pd.DataFrame(profit.values(), profit.keys(), columns=["Test 1 profit/" + START_DATE_TEST1, "Teste operações"]).T
concated = pd.concat([ concated, alo ])

# test(START_DATE_TEST2, END_DATE_TEST2)
# alo = pd.DataFrame(profit.values(), profit.keys(), columns=["Test 2 profit/" + START_DATE_TEST2]).T
# concated = pd.concat([ concated, alo ])

# test(START_DATE_TEST3, END_DATE_TEST3)
# alo = pd.DataFrame(profit.values(), profit.keys(), columns=["Test 3 profit/" + START_DATE_TEST3]).T
# concated = pd.concat([ concated, alo ])

# test(START_DATE_TEST4, END_DATE_TEST4)
# alo = pd.DataFrame(profit.values(), profit.keys(), columns=["Test 4 profit/" + START_DATE_TEST4]).T
# concated = pd.concat([ concated, alo ])

concated = concated.T
concated_mean = concated.mean()

concated.loc['Média'] = concated_mean

import os

n = 0
file = 'TEST'
extention = '.xlsx'

while True:
    fileName = file + str(n) + extention
    if os.path.exists(fileName):
        n += 1
    else:
        concated.to_excel(fileName)
        break
